# Convolutional Neural Network

## TensorFlow Keras, Layers Conv2D

In [25]:
import tensorflow as tf

In [26]:
# The inputs are 28x28 RGB images with `channels_last` and the batch
# size is 4.
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
    2, 3, strides=1, activation='relu', input_shape=input_shape[1:])(x)
print(y.shape)


In [27]:
# With `dilation_rate` as 2.
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
    2, 3,
    activation='relu',
    dilation_rate=2,
    input_shape=input_shape[1:])(x)
print(y.shape)


In [28]:
 # With `padding` as "same".
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
    2, 3, activation='relu', padding="same", input_shape=input_shape[1:])(x)
print(y.shape)


In [29]:
# With extended batch shape [4, 7]:
# input_shape = (4, 7, 28, 28, 3)
# x = tf.random.normal(input_shape)
# y = tf.keras.layers.Conv2D(
#     2, 3, activation='relu', input_shape=input_shape[2:])(x)
# print(y.shape)


## TensorFlow Keras, Layers Conv3D

In [41]:
# The inputs are 28x28x28 volumes with a single channel, and the
# batch size is 4
input_shape =(4, 28, 28, 28, 1)
x = tf.random.normal(input_shape)
x = tf.keras.layers.Conv3D(
    2, 3, activation='relu', input_shape=input_shape[1:])(x)
print(x.shape)
x = tf.keras.layers.Conv3D(
    4, 3, activation='relu')(x)
print(x.shape)
y = tf.keras.layers.GlobalAveragePooling3D()(x)
print(y.shape)


In [31]:
# # With extended batch shape [4, 7], e.g. a batch of 4 videos of
# # 3D frames, with 7 frames per video.
# input_shape = (4, 7, 28, 28, 28, 1)
# x = tf.random.normal(input_shape)
# y = tf.keras.layers.Conv3D(
#     2, 3, activation='relu', input_shape=input_shape[2:])(x)
# print(y.shape)


# Try to build a 3D convolutional neural network for the Landmark Detection project

## Load Data

In [ ]:
import pandas as pd
import numpy as np

ROI_CT_path = '../Resources/ROI_CT.xlsx'
ROI_MR_path = '../Resources/ROI_MR.xlsx'

roi_CT = pd.read_excel(ROI_CT_path, index_col=[0, 1], header=[0, 1])

In [ ]:
pname_list = list(set([x for (x, y) in roi_CT.index]))
pname_list.sort()

print(pname_list)

In [ ]:
pixel_space = [0.15, 0.15, 0.15]
data_path_base = "/Volumes/Shawn_HDD/PhD/Project/Date/CT_Augmented/"

p_name = pname_list[0]
data_path = data_path_base + p_name + "_aug_0.npy"
volume_data = np.load(data_path, allow_pickle=True).item()

In [ ]:
import functions

functions.show_pts(volume_data.get("volume"), volume_data.get("pts"), pixel_space)

## Load data from .mat --- train, test, valid

In [32]:
from os import listdir
from os.path import isfile, join
import numpy as np
import h5py

In [33]:

# e.g. AZ_17017030_AugVol_1.mat
X_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Input/"
# e.g. AZ_17017030_AugPts_1.mat
Y_train_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Train/Output/"

X_train_files = [f for f in listdir(X_train_base_path) if isfile(join(X_train_base_path, f))]

X_train = []
Y_train = []
for X_train_file in X_train_files:
    X_train_file_path = join(X_train_base_path, X_train_file)
    Y_train_file_path = join(Y_train_base_path, X_train_file.replace("AugVol", "AugPts"))
    file_vol = h5py.File(X_train_file_path, 'r')
    file_pts = h5py.File(Y_train_file_path, 'r')
    load_mat_vol = file_vol.get('rescaled_aug_vol')
    load_mat_pts = file_pts.get('rescaled_aug_pts')
    X_train.append(np.array(load_mat_vol).T)
    Y_train.append(np.array(load_mat_pts).reshape(3,4).T)
    file_vol.close()
    file_pts.close()

In [46]:

# e.g. AZ_17017030_AugVol_1.mat
X_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Input/"
# e.g. AZ_17017030_AugPts_1.mat
Y_val_base_path = "/Volumes/Shawn_SSD/PhD/Project/Date/augmentation_from_matlab/Val/Output/"

X_val_files = [f for f in listdir(X_val_base_path) if isfile(join(X_val_base_path, f))]

X_val = []
Y_val = []
for X_val_file in X_val_files:
    X_val_file_path = join(X_val_base_path, X_val_file)
    Y_val_file_path = join(Y_val_base_path, X_val_file.replace("AugVol", "AugPts"))
    file_vol = h5py.File(X_val_file_path, 'r')
    file_pts = h5py.File(Y_val_file_path, 'r')
    load_mat_vol = file_vol.get('rescaled_aug_vol')
    load_mat_pts = file_pts.get('rescaled_aug_pts')
    X_val.append(np.array(load_mat_vol).T)
    Y_val.append(np.array(load_mat_pts).reshape(3,4).T)
    file_vol.close()
    file_pts.close()

In [47]:
print(np.shape(X_train))
print(np.shape(Y_train))

print(np.shape(X_val))
print(np.shape(Y_val))

In [50]:
X_train_reshape = np.asarray(X_train).reshape(700, 170, 170, 30, 1)
Y_train = np.asarray(Y_train)[:, 0, :]

In [51]:
X_val_reshape = np.asarray(X_val).reshape(100, 170, 170, 30, 1)
Y_val = np.asarray(Y_val)[:, 0, :]

In [53]:
print(Y_train.shape)
print(X_train_reshape.shape)

print(Y_val.shape)
print(X_val_reshape.shape)

In [57]:
import tensorflow as tf
from tensorflow import keras

train_dataset = tf.data.Dataset.from_tensor_slices((X_train_reshape, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=1400).batch(2)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val_reshape, Y_val))
val_dataset = val_dataset.shuffle(buffer_size=200).batch(2)

In [58]:
for step, (x, y) in enumerate(train_dataset):
    print("step: ", step)
    print("x shape: ", x.shape, type(x))
    print("y shape: ", y.shape, type(y))
    print(y.shape)
    break


for step, (x, y) in enumerate(val_dataset):
    print("step: ", step)
    print("x shape: ", x.shape, type(x))
    print("y shape: ", y.shape, type(y))
    print(y.shape)
    break

In [ ]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((X_train_reshape, Y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((X_val_reshape, Y_val))

batch_size = 2
train_dataset = (
    train_loader.shuffle(len(X_train_reshape))
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(X_val))
    .batch(batch_size)
    .prefetch(2)
)

## Build model

In [54]:
import tensorflow.keras.layers as layers

def get_model(width=170, height=170, depth=30):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=3, )(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

# Build model.
model = get_model(width=170, height=170, depth=30)
model.summary()

In [61]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="mean_squared_error",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    #metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)